<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/czasy-joinow/colabs/Czasy_joinow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [3]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 200 kB 52.4 MB/s 
     |████████████████████████████████| 199 kB 55.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=f91307b57cef325e0227fff1957726e7195f3eb53fa551df87a3a0ff7f8eeea7
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-20 18:25:26--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [4]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [6]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-20 18:29:40--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.73, 13.249.98.61, 13.249.98.91, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236619296 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.66M  90.6MB/s    in 2.5s    

2022-11-20 18:29:42 (90.6 MB/s) - ‘name.basics.tsv.gz’ saved [236619296/236619296]

--2022-11-20 18:29:43--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.73, 13.249.98.61, 13.249.98.91, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287876715 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.54M  93.2M

##Rozpakowanie danych

In [7]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [8]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [9]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0031983,tt00504...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0038355,tt01170...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00544...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [10]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [9]:
#df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

## Sprawdźmy czasy joinowania dla różnych kombinacji tabel

Sprawdzimy najpierw jak długo zajmuje łączenie tabel bez usuniętych niepotrzebnych rekordów. Rozważmy różne kombinacje łączeń, gdzie tabele są łączone w następujący sposób:
* df_name_basics_selected i df_title_principals_selected, typ łączenia "inner",
* df_name_basics_selected i df_title_principals_selected, typ łączenia "right",
* dane wcześniej złączone i df_title_basic_selected, typ łączenia "inner",
* dane wcześniej złączone i df_title_basic_selected, typ łączenia "left",
* df_title_principals_selected i df_name_basics_selected, typ łączenia "inner",
* df_title_principals_selected i df_name_basics_selected, typ łączenia "left",
* df_title_basic_selected i df_title_principals_selected, typ łączenia "inner",
* df_title_basic_selected i df_title_principals_selected, typ łączenia "right",
* dane wcześniej złączone i df_name_basics_selected, typ łączenia "inner",
* dane wcześniej złączone i df_name_basics_selected, typ łączenia "right",
* df_title_principals_selected i df_title_basic_selected, typ łączenia "inner",
* df_title_principals_selected i df_title_basic_selected, typ łączenia "left".

In [10]:
from timeit import default_timer as timer

start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53129560, 8)
joined table size:  (53123422, 12) time of joining tables:  146.88740994900002


In [11]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53129560, 8)
joined table size:  (53129560, 12) time of joining tables:  140.78069022199998


In [12]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53137392, 8)
joined table size:  (53131216, 12) time of joining tables:  133.27368778899995


In [13]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (53137392, 8)
joined table size:  (53137392, 12) time of joining tables:  136.10786328500012


In [14]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53129560, 8)
joined table size:  (53123422, 12) time of joining tables:  135.32172029000003


In [15]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53129560, 8)
joined table size:  (53129560, 12) time of joining tables:  134.1102023620001


In [16]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53137392, 8)
joined table size:  (53131216, 12) time of joining tables:  134.41051492299994


In [17]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (53137392, 8)
joined table size:  (53137392, 12) time of joining tables:  144.114738408


In [18]:
start = timer()
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data9.count(), len(data9.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53131216, 8)
joined table size:  (53123422, 12) time of joining tables:  119.14489808400003


In [19]:
start = timer()
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data10.count(), len(data10.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53131216, 8)
joined table size:  (53131216, 12) time of joining tables:  119.46920445899968


In [20]:
start = timer()
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data11.count(), len(data11.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53137392, 8)
joined table size:  (53129560, 12) time of joining tables:  117.28335163100019


In [21]:
start = timer()
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data12.count(), len(data12.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (53137392, 8)
joined table size:  (53137392, 12) time of joining tables:  117.9556835229996


In [22]:
start = timer()
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data13.count(), len(data13.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53131216, 8)
joined table size:  (53123422, 12) time of joining tables:  128.463416349


In [23]:
start = timer()
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data14.count(), len(data14.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53131216, 8)
joined table size:  (53131216, 12) time of joining tables:  119.23630927600061


In [24]:
start = timer()
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data15.count(), len(data15.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53137392, 8)
joined table size:  (53129560, 12) time of joining tables:  114.516300968


In [25]:
start = timer()
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("joined table size: ", (data16.count(), len(data16.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (53137392, 8)
joined table size:  (53137392, 12) time of joining tables:  118.9455270540002


Teraz zmierzymy czasy tych samych złączeń, jednak po usunięciu z tabel niepotrzebnych rekordów.

In [11]:
df_name_basics = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

In [27]:
start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20175420, 8)
joined table size:  (20171936, 12) time of joining tables:  99.12242422700001


In [28]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20175420, 8)
joined table size:  (20175420, 12) time of joining tables:  100.22442791100002


In [29]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20798916, 8)
joined table size:  (20795363, 12) time of joining tables:  94.53604484700008


In [30]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20798916, 8)
joined table size:  (20798916, 12) time of joining tables:  95.53096150900001


In [31]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (20175420, 8)
joined table size:  (20171936, 12) time of joining tables:  96.12059715300074


In [32]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (20175420, 8)
joined table size:  (20175420, 12) time of joining tables:  96.411182414


In [33]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
end = timer()
print("joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (20798916, 8)
joined table size:  (20795363, 12) time of joining tables:  96.13773970000057


In [34]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
end = timer()
print("joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_name_basics_selected joined table size:  (20798916, 8)
joined table size:  (20798916, 12) time of joining tables:  95.82845264200023


In [35]:
start = timer()
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data9.count(), len(data9.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (20795363, 8)
joined table size:  (20171936, 12) time of joining tables:  96.3941217419997


In [36]:
start = timer()
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "right")
end = timer()
print("joined table size: ", (data10.count(), len(data10.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (20795363, 8)
joined table size:  (22543035, 12) time of joining tables:  101.68330978599988


In [37]:
start = timer()
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data11.count(), len(data11.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (20798916, 8)
joined table size:  (20175420, 12) time of joining tables:  92.04867105800076


In [38]:
start = timer()
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "right")
end = timer()
print("joined table size: ", (data12.count(), len(data12.columns)), "time of joining tables: ", end-start)

df_title_basic_selected and df_title_principals_selected joined table size:  (20798916, 8)
joined table size:  (22546438, 12) time of joining tables:  93.66210862499975


In [39]:
start = timer()
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data13.count(), len(data13.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (20795363, 8)
joined table size:  (20171936, 12) time of joining tables:  95.18693754500055


In [40]:
start = timer()
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "right")
end = timer()
print("joined table size: ", (data14.count(), len(data14.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (20795363, 8)
joined table size:  (22543035, 12) time of joining tables:  95.27838813699964


In [41]:
start = timer()
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("joined table size: ", (data15.count(), len(data15.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (20798916, 8)
joined table size:  (20175420, 12) time of joining tables:  102.57129950200033


In [42]:
start = timer()
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "right")
end = timer()
print("joined table size: ", (data16.count(), len(data16.columns)), "time of joining tables: ", end-start)

df_title_principals_selected and df_title_basic_selected joined table size:  (20798916, 8)
joined table size:  (22546438, 12) time of joining tables:  107.62590241399994


Sprawdzimy jakie braki mają nasze dane.

In [44]:
print(data11.filter(data11.primaryName.isNull()).count())
print(data11.filter(data11.category.isNull()).count())
print(data11.filter(data11.genres.isNull()).count())

0
0
3484


Po kilku uruchomieniach kodu, za każdym razem wychodzą różne wyniki, dlatego ciężko stwierdzić, który sposób łączenia danych jest najszybszy, jednak na pewno widać, że łączenie tabel bez zbędnych rekordów jest szybsze niż łączenie pełnych tabel. W ostatecznym rozrachunku, najlepiej wypada sposób łaczenia:
* df_title_basic_selected i df_title_principals_selected, typ łączenia "right"
* wcześniej złączone dane i df_name_basics_selected, typ łączenia "inner".

W danch tych nie ma żadnych nadmiarowych aktorów, pojawiają się jedynie braki co do filmów, gdyż nie wszystkie filmy, w których grali aktorzy pojawiają się w bazie.

Ten sposób łączenia danych zostanie zastosowany w notatniku głównym.


In [12]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_name_basics_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (4426315, 5)
df_title_principals_selected dataframe size:  (20798916, 4)
df_title_basic_selected dataframe size:  (9386465, 5)
joined dataframe size:  (20798916, 8)
joined dataframe size:  (20175420, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [13]:
sample = data.rdd.takeSample(False, 5)

In [14]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,nconst,tconst,titleType,originalTitle,isAdult,genres,category,characters,primaryName,birthYear,deathYear,knownForTitles
0,nm0467502,tt1199515,tvMovie,Precek,0,Drama,actor,"[""Martin""]",Dragi Kostovski,1930,1980,"tt1247355,tt0182313,tt1369730,tt0182367"
1,nm0862117,tt1729191,tvMovie,Familienanschluß,0,Comedy,actor,"[""Gerichtsvollzieher""]",Hanno Thurau,1939,1992,"tt0108574,tt0080612,tt0806910,tt0131686"
2,nm0681566,tt0252262,short,Le bateau de Lu,0,Short,actor,"[""Un client""]",Michel Piccoli,1925,2020,"tt0118613,tt1456472,tt0057345,tt0061395"
3,nm5379363,tt16254088,tvEpisode,Episode #1.64,0,Drama,actress,"[""Ivie"",""Alero""]",Ivie Okujaye,\N,\N,"tt6911110,tt18182222,tt2526570,tt4452274"
4,nm4049315,tt4359146,tvEpisode,Episode #1.28,0,\N,actress,"[""Anastasia""]",Marina Shtoda,1982,\N,"tt6461726,tt2105514,tt4353482,tt12099306"
